<a href="https://colab.research.google.com/github/rshriroop01/Expenses/blob/master/credit2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ipo_stock_2010_2018_v2.csv to ipo_stock_2010_2018_v2.csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, accuracy_score,fbeta_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [ ]:
df = pd.read_csv('ipo_stock_2010_2018_v2.csv', parse_dates=['Date Priced', 'year'])

In [ ]:
print(df.shape)
df.head()

(834, 26)


,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,descriptions,link_nasdaq,year,firstday_adjclose,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,inyear_adjclose,inyear_open,sector,industry,employees2019,CEO_pay,CEO_born
0,ANDATEE CHINA MARINE FUEL SERVICES CORP,AMCF,NASDAQ,6.3,3134921.0,19750002.0,2010-01-26,128.0,NO. 68 BINHAI RD DALIAN XIGANG DISTRICTDALIAN ...,NaN,"We, through our VIE entities are engaged in th...",https://www.nasdaq.com/markets/ipos/company/an...,2010-01-01,5.770000,6.30,7.590000,8.000000,5.830000,6.030000,2.720000,2.58,Energy,Oil & Gas Equipment & Services,189.0,NaN,NaN
1,GENERAC HOLDINGS INC.,GNRC,New York Stock Exchange,13.0,18750000.0,243750000.0,2010-02-11,1486.0,"S45 W29290 HIGHWAY 59WAUKESHA, WI 53187",WI,We are a leading designer and manufacturer of ...,https://www.nasdaq.com/markets/ipos/company/ge...,2010-01-01,8.460629,13.00,9.719180,14.910000,8.612183,13.050000,13.415763,20.24,Industrials,Diversified Industrials,5046.0,2520000.0,1972.0
2,"QUINSTREET, INC",QNST,NASDAQ,15.0,10000000.0,150000000.0,2010-02-11,568.0,"950 TOWER LANE, 6TH FLOORFOSTER CITY, CA 94404",CA,QuinStreet is a leader in vertical marketing a...,https://www.nasdaq.com/markets/ipos/company/qu...,2010-01-01,15.000000,15.00,16.410000,16.010000,12.980000,13.500000,12.740000,12.52,Technology,Internet Content & Information,506.0,816150.0,1960.0
3,TERRENO REALTY CORP,TRNO,New York Stock Exchange,20.0,8750000.0,175000000.0,2010-02-10,6.0,"16 MAIDEN LANEFIFTH FLOORSAN FRANCISCO, CA 94108",CA,"We are an internally managed, newly organized ...",https://www.nasdaq.com/markets/ipos/company/te...,2010-01-01,14.692022,18.75,15.330121,19.530001,14.770803,18.809999,13.518149,17.02,Real Estate,REIT - Industrial,23.0,799500.0,1961.0
4,"PIEDMONT OFFICE REALTY TRUST, INC.",PDM,New York Stock Exchange,14.5,12000000.0,174000000.0,2010-02-10,109.0,"5565 GLENRIDGE CONNECTORSTE 450ATLANTA, GA 30342",GA,"We are a fully integrated, self-administered a...",https://www.nasdaq.com/markets/ipos/company/pi...,2010-01-01,9.543714,14.75,12.046385,19.299999,10.400203,17.830000,14.289856,21.02,Real Estate,REIT - Office,134.0,NaN,NaN


In [ ]:
df.columns

Index(['Company Name', 'Symbol', 'Market', 'Price', 'Shares', 'Offer Amount',
       'Date Priced', 'employees', 'address', 'US_state', 'descriptions',
       'link_nasdaq', 'year', 'firstday_adjclose', 'firstday_open',
       'inmonth_adjclose', 'inmonth_open', 'inweek_adjclose', 'inweek_open',
       'inyear_adjclose', 'inyear_open', 'sector', 'industry', 'employees2019',
       'CEO_pay', 'CEO_born'],
      dtype='object')

In [ ]:
keep_cols = ['Company Name', 'Price', 'Shares', 'Offer Amount',
       'Date Priced', 'employees', 'US_state', 'firstday_adjclose', 'firstday_open',
       'inmonth_adjclose', 'inweek_adjclose', 'sector', 'CEO_pay', 'CEO_born']

In [ ]:
df = df[keep_cols]

In [ ]:
df['US_state'] = df['US_state'].apply(lambda x: 'foreign' if str(x) == 'nan' else x)

In [ ]:
df['diff_close_inday_2_open_firstday'] = df['firstday_adjclose'] - df['firstday_open']

In [ ]:
df['check_prices'] = df['firstday_open']/df['Price']

In [ ]:
typo_prices = df[(df['check_prices'] > 2) | (df['check_prices'] < -2)][['Company Name', 'Price', 'firstday_open','firstday_adjclose', 'inweek_adjclose']]

In [ ]:
print(typo_prices.shape[0])
typo_prices.head()

47


,Company Name,Price,firstday_open,firstday_adjclose,inweek_adjclose
7,SCORPIO TANKERS INC.,13.0,127.500000,121.557602,120.686592
23,"ROADRUNNER TRANSPORTATION SYSTEMS, INC.",14.0,343.750000,343.750000,348.750000
36,"AMYRIS, INC.",16.0,247.500000,247.500000,256.049988
43,DAQO NEW ENERGY CORP.,9.5,49.900002,51.250000,67.000000
47,GLOBUS MARITIME LTD,11.6,476.000000,488.170715,497.734833


In [ ]:
df = df.drop(typo_prices.index, axis=0)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isna().sum()

Company Name                          0
Price                                 1
Shares                                1
Offer Amount                          0
Date Priced                           0
employees                            14
US_state                              0
firstday_adjclose                     0
firstday_open                         0
inmonth_adjclose                      0
inweek_adjclose                       0
sector                               10
CEO_pay                             214
CEO_born                             70
diff_close_inday_2_open_firstday      0
check_prices                          1
dtype: int64

In [ ]:
#drop 1 row where there is no Offer Price
df.drop(df[df.Price.isna()].index, axis=0, inplace=True)

In [ ]:
#drop 14 rows where there are no employees
df.drop(df[df.employees.isna()].index, axis=0, inplace=True)

In [ ]:
#drop 10 rows where there is no sector
df.drop(df[df.sector.isna()].index, axis=0, inplace=True)

In [ ]:
df.isna().sum()

Company Name                          0
Price                                 0
Shares                                0
Offer Amount                          0
Date Priced                           0
employees                             0
US_state                              0
firstday_adjclose                     0
firstday_open                         0
inmonth_adjclose                      0
inweek_adjclose                       0
sector                                0
CEO_pay                             198
CEO_born                             58
diff_close_inday_2_open_firstday      0
check_prices                          0
dtype: int64

In [ ]:
#drop rows where there is no CEO_born
df.drop(df[df.CEO_born.isna()].index, axis=0, inplace=True)

In [ ]:
#drop rows where there is no CEO_pay
df.drop(df[df.CEO_pay.isna()].index, axis=0, inplace=True)

In [ ]:
df['month'] = df['Date Priced'].map(lambda x: x.month)

In [ ]:
df['month'] = df['month'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug',\
                                  9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})

In [ ]:
df['weekday'] = df['Date Priced'].dt.dayofweek.map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'})

In [ ]:
df.shape

(565, 18)

In [ ]:
df.head()

,Company Name,Price,Shares,Offer Amount,Date Priced,employees,US_state,firstday_adjclose,firstday_open,inmonth_adjclose,inweek_adjclose,sector,CEO_pay,CEO_born,diff_close_inday_2_open_firstday,check_prices,month,weekday
1,GENERAC HOLDINGS INC.,13.0,18750000.0,243750000.0,2010-02-11,1486.0,WI,8.460629,13.00,9.719180,8.612183,Industrials,2520000.0,1972.0,-4.539371,1.000000,Feb,Thu
2,"QUINSTREET, INC",15.0,10000000.0,150000000.0,2010-02-11,568.0,CA,15.000000,15.00,16.410000,12.980000,Technology,816150.0,1960.0,0.000000,1.000000,Feb,Thu
3,TERRENO REALTY CORP,20.0,8750000.0,175000000.0,2010-02-10,6.0,CA,14.692022,18.75,15.330121,14.770803,Real Estate,799500.0,1961.0,-4.057978,0.937500,Feb,Wed
6,SS&C TECHNOLOGIES HOLDINGS INC,15.0,10725000.0,160875000.0,2010-03-31,1253.0,CT,7.181165,8.00,7.957377,7.862139,Technology,10880000.0,1955.0,-0.818835,0.533333,Mar,Wed
8,FIRST INTERSTATE BANCSYSTEM INC,14.5,10000000.0,145000000.0,2010-03-24,1730.0,MT,11.994637,16.00,12.239342,12.331657,Financial Services,1710000.0,1960.0,-4.005363,1.103448,Mar,Wed


Special condition

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving spy.csv to spy.csv


In [ ]:
sp = pd.read_csv('spy.csv')

In [ ]:
# we need to have our Date values ordered 
#even if it is oredered already, let's do it anyway as a good practice working with dates
sp.sort_values('Date', inplace=True)
sp.reset_index(drop=True, inplace=True)
sp.Date = pd.to_datetime(sp.Date)
sp.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
1,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2,2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
3,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
4,2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


In [ ]:
def get_week_chg(ipo_dt):
    try: 
        day_ago_idx = sp[sp['Date'] == str(ipo_dt.date())].index[0] - 1
        week_ago_idx = sp[sp['Date'] == str(ipo_dt.date())].index[0] - 8
        chg = (sp.iloc[day_ago_idx]['Close'] - sp.iloc[week_ago_idx]['Close'])/(sp.iloc[week_ago_idx]['Close'])
        return chg * 100
    except:
        print('error', ipo_dt.date())      

In [ ]:
df['SP_Week_Chg'] = df['Date Priced'].map(get_week_chg)

In [ ]:
X = dmatrix("Q('Price') + Q('Shares') + Q('Offer Amount') + Q('employees') + \
            Q('sector') + Q('SP_Week_Chg') + Q('CEO_pay') + Q('CEO_born')+ Q('weekday')", data=df, return_type='dataframe')

In [ ]:
X.shape

(565, 22)

In [ ]:
#predict success if IPO's price is more than $1 change on the first trading day
y = df['diff_close_inday_2_open_firstday'].apply(lambda x: 1 if x >= 1 else 0)
print('number of true positives -', y.sum())

number of true positives - 101


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalize

In [ ]:
# Initialize a scaler, then apply it to the features
scaler = StandardScaler() # default=(0, 1)
numerical = ["Q('Price')", "Q('Shares')", "Q('Offer Amount')", "Q('employees')", "Q('SP_Week_Chg')", "Q('CEO_pay')","Q('CEO_born')"]

X_train.loc[:, numerical] = scaler.fit_transform(X_train.loc[:, numerical])
X_test.loc[:, numerical] = scaler.fit_transform(X_test.loc[:, numerical])

# Show an example of a record with scaling applied
X_train.head()

,Intercept,Q('sector')[T.Communication Services],Q('sector')[T.Consumer Cyclical],Q('sector')[T.Consumer Defensive],Q('sector')[T.Energy],Q('sector')[T.Financial Services],Q('sector')[T.Healthcare],Q('sector')[T.Industrials],Q('sector')[T.Real Estate],Q('sector')[T.Technology],Q('sector')[T.Utilities],Q('weekday')[T.Mon],Q('weekday')[T.Thu],Q('weekday')[T.Tue],Q('weekday')[T.Wed],Q('Price'),Q('Shares'),Q('Offer Amount'),Q('employees'),Q('SP_Week_Chg'),Q('CEO_pay'),Q('CEO_born')
463,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019169,-0.053192,-0.082862,-0.160494,0.809244,0.150742,-0.142372
165,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.202128,-0.124432,-0.030858,1.014218,0.819415,-0.308502,1.046888
294,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.149825,-0.278163,-0.201165,-0.159531,2.503508,-0.355672,-0.406652
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.318819,0.199901,-0.004301,-0.082393,-1.330538,1.071479,1.179028
678,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.357157,-0.090687,-0.074236,-0.113826,-0.195258,-0.294281,1.179028


Model

In [ ]:
clf = LogisticRegression(solver='lbfgs', class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
#evaluate the model on hold-out data
print('accuracy score:', clf.score(X_test, y_test))
print('recall score:', recall_score(y_test, y_pred))
print('precision score:', precision_score(y_test, y_pred))
print('F-beta score:', fbeta_score(y_test, y_pred, average='macro', beta=0.1))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('odds_ratio:', (tp/fp)/(fn/tn))

accuracy score: 0.6548672566371682
recall score: 0.6296296296296297
precision score: 0.3695652173913043
F-beta score: 0.6097235272074857
odds_ratio: 3.3413793103448275


AdaBOOST

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=43, sampling_strategy=1.0)
oversample = SMOTE()

X_sm, y_sm = oversample.fit_resample(X_train, y_train)
print(X_sm.shape, y_sm.shape)

(756, 22) (756,)


In [ ]:
clf = AdaBoostClassifier(n_estimators=1000, learning_rate=0.05)

clf.fit(X_sm, y_sm)
y_pred = clf.predict(X_test)

#evaluate the model on hold-out data
print('accuracy score:', clf.score(X_test, y_test))
print('recall score:', recall_score(y_test, y_pred))
print('precision score:', precision_score(y_test, y_pred))
print('F-beta score:', fbeta_score(y_test, y_pred, average='macro', beta=0.1))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('odds_ratio:', (tp/fp)/(fn/tn))

accuracy score: 0.7256637168141593
recall score: 0.1111111111111111
precision score: 0.3
F-beta score: 0.5316398976151515
odds_ratio: 1.4107142857142856


In [ ]:
from catboost import CatBoostClassifier
clf_ = CatBoostClassifier()

clf_.fit(X_sm, y_sm)
y_pred = clf_.predict(X_test)

#evaluate the model on hold-out data
print('accuracy score:', clf_.score(X_test, y_test))
print('recall score:', recall_score(y_test, y_pred))
print('precision score:', precision_score(y_test, y_pred))
print('F-beta score:', fbeta_score(y_test, y_pred, average='macro', beta=0.1))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('odds_ratio:', (tp/fp)/(fn/tn))

In [ ]:
importance = clf_.feature_importances_
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
r_auc = roc_auc_score(y_test, y_pred)

In [ ]:
print('Random (chance) Prediction: AUROC = %.3f' % (r_auc))
print('Random Forest: AUROC = %.3f' % (rf_auc))
print('Naive Bayes: AUROC = %.3f' % (nb_auc))

In [ ]:
importance = clf.feature_importances_
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
def create_results_df(X_train, X_test, y_pred, y_test):
    
    results = []
    for idx, chg, ypred, ytrue in zip(y_test.index, df['diff_close_inday_2_open_firstday'][y_test.index], y_pred, y_test):
        if ypred == ytrue:
            results.append([idx, chg, ypred, ytrue, 1])
        else:
            results.append([idx, chg, ypred, ytrue, 0])
    return pd.DataFrame(results, columns=['index', '$_chg', 'predicted', 'actual', 'correct'])

In [ ]:
rf_log = create_results_df(X_train, X_test, y_pred, y_test)

In [ ]:
def return_model_keyresults(results_df):
    summ_pred = results_df[results_df.predicted == 1]['$_chg'].sum()
    mean_pred = results_df[results_df.predicted == 1]['$_chg'].mean()
    total_pred = results_df.predicted.sum()
    
    summ_actual = results_df[results_df.actual == 1]['$_chg'].sum()
    mean_actual = results_df[results_df.actual == 1]['$_chg'].mean()
    total_actual = results_df.actual.sum()
    
    summ_naive = results_df['$_chg'].sum()
    mean_naive = results_df['$_chg'].mean()
    total_naive = len(results_df['$_chg'])
    print('actual successes:', results_df.actual.sum())
    print('predicted successes:', results_df.predicted.sum())
    print('predicted gain: {:0.2f} for {} IPOs -- {:0.2f} per IPO '.format(summ_pred, total_pred, mean_pred))
    print('actual gain: {:0.2f} for {} IPOs -- {:0.2f} per IPO '.format(summ_actual, total_actual, mean_actual))
    print('naive gain: {:0.2f} for {} IPOs -- {:0.2f} per IPO '.format(summ_naive, total_naive, mean_naive))

In [ ]:
return_model_keyresults(rf_log)

actual successes: 27
predicted successes: 46
predicted gain: $37.62 for 46 IPOs -- $0.82 per IPO 
actual gain: $85.99 for 27 IPOs -- $3.18 per IPO 
naive gain: $-34.09 for 113 IPOs -- $-0.30 per IPO 
